# Exercise Set 1

## Multiple Choice / True-False

Correct answers are `marked like this`.

**Q1.** When dealing with numerical instability, which of the following is NOT a recommended approach?

> a. Using well-scaled and precise data

> b. Ensuring matrix coefficients are within 6 orders of magnitude

> c. `Increasing the Big M value as much as possible`

> d. Setting the NumericFocus parameter to prioritize stability

**Q2.** Which of the following statements about MIP Gap is accurate?

> a. A lower MIP Gap always leads to significant solution improvements

> b. `A higher MIP Gap can still yield a solution close to optimal, depending on the problem`

> c. Setting MIP Gap to zero is ideal for all real-world applications

> d. MIP Gap only affects runtime but not solution quality

**Q3.** Which of the following is a potential benefit of using the Big M method in an optimization model?

> a. `It helps establish conditional constraints based on binary variables`

> b. It eliminates the need for slack variables

> c. It reduces the numerical range required for coefficients

> d. It ensures the model is solved to exact optimality

**Q4.** An optimization model can sometimes be infeasible in exact arithmetic but feasible within the solver’s tolerances.

> `True` 

> False

**Q5.** For which of the following application areas is non-linear optimization used in practice?

> a. Finance

> b. Utilities

> c. Engineering

> d. `All of the above`

**Q6.** Which type of optimization problem could have local optimal solutions that are not globally optimal?

> a. Linear programs

> b. Convex Quadratic Programs

> c. `Non-convex Quadratic Programs`

> d. None of the above

**Q7.** Which of the following constraints defines a convex feasible region?

> a. `Correct:` $y \geq x^2$

> b. $y \leq x^2$

> c. $y = x^2$

> d. $y \neq x^2$

## Simple modeling / coding

**Q8.** Show these constraints are either compatible (find a feasible solution) or incompatible (prove a contradiction) algebraically:
> **Q8a.** 

\begin{align*}
x + y + z &\ge 3   \quad &(1)    \\
2x + 3y - z &\le 4   \quad &(2)  \\
x, y, z &\ge 0
\end{align*}

`Solution:`
An example solution is to select $x = y = z = 1$.

> **Q8b.** 

\begin{align*}
x_1 + x_2 + x_3 &= 6 \quad &(1)      \\
3x_1 + 2x_2 + x_3 &\ge 20  \quad &(2)\\
x_1, x_2, x_3, &\ge 0
\end{align*}

`Solution:`

One way to derive a contradiction:
Multiply $(1)$ by 3, which gives $3x_1 + 3x_2 + 3x_3 = 18$. 

Since $x_1, x_2, x_3$ are all nonnegative,
\begin{align*}
3x_1 + 3x_2 + 3x_3 \ge 3x_1 + 2x_2 + x_3 \ge 20
\end{align*}

This reduces to:

\begin{align*}
18 \ge 3x_1 + &2x_2 + x_3 \ge 20 \\
18 &\ge 20
\end{align*}

Which is a contradiction. 

**Q9.** Translate the following into constraints using binary variables for deciding to invest in three stocks. Then determine if they are compatible as above. 

- I will invest in exactly two of the three stocks.
- If I invest in stock A, then I will invest in stock B.
- If I invest in stock B, then I will invest in stock C.

`Solution:`
Let $y_A, y_B, y_C \in \{0,1\}$ be the three stocks.
\begin{align*}
\sum_{i \in \{A,B,C\}} y_i &= 2 \\
y_A &\le y_B \\
y_B &\le y_C \\
\end{align*}
$y_A = 0, y_B = 1, y_C = 1$ satisfies the constraints. 


## Back to the widget production problem
Use the code below as the base model for this problem. 

In [1]:
%pip install 'gurobipy>=12.0.0'

# Import packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

Note: you may need to restart the kernel to use updated packages.


In [2]:
# read in transportation cost data
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv')

# get production and distribution locations from data frame
production = list(transp_cost['production'].unique())
distribution = list(transp_cost['distribution'].unique())
transp_cost = transp_cost.set_index(['production','distribution']).squeeze()

max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")

# the min production is a fraction of the max
frac = 0.75
C = 30

In [3]:
# gurobipy code for model
m = gp.Model('widgets')
m.setParam('OutputFlag',0)

# decision variables - reminder that we ended on x being a semi-continuous variable type
x = m.addVars(production, distribution, vtype=GRB.SEMICONT, lb = C, name = 'prod_ship')

# constraints
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
meet_demand = m.addConstrs(x.sum('*', d) >= n_demand[d] for d in distribution)

#objective
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

m.optimize()

x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)

print(f"The original model had a total cost of {round(m.ObjVal,2)}")
sol[sol.shipment > 0]

Restricted license - for non-production use only - expires 2026-11-23
The original model had a total cost of 1748.42


cost  shipment
production  distribution                
Baltimore   Nashville     5.96      30.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53      71.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      41.0

## Nonlinear optimization modeling

### Quadratic expression

In order to produce more widgets, production facilities must run their machines at faster speeds. This consumes more energy and produces more waste at a level that grows with each widget produced. Running their machines costs a production facility $0.01 times the square of the number of widgets produced. Modify the objective function to include this additional production cost. Here is the some code to get you started - it includes the original transportation cost expression that you will be adding to:

```python
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution) + ..., GRB.MINIMIZE)
```
Solve the model with the updated objective and see how the cost has changed.

In [4]:
# One solution
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution) + 0.01 * gp.quicksum(x.sum(p, '*') * x.sum(p, '*') for p in production), GRB.MINIMIZE)
m.optimize()
print(f"The updated model had a total cost of {round(m.ObjVal,2)}")

# Another solution
# Introduces auxiliary variables representing total production for each facility
# produced = m.addVars(production, name='total_produced')
# m.addConstrs((produced[p] == x.sum(p, '*') for p in production), name='set_produced')
# m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution) + 0.01 * gp.quicksum(produced[p] * produced[p] for p in production), GRB.MINIMIZE)

The updated model had a total cost of 2808.99


### Nonlinear expression 1

The widgets will now be shipped using hyper-efficient vehicles. The cost to ship $x_{ij}$ widgets from production facility $i$ to distribution center $j$ is now equal to the corresponding transportation cost times $\log(1 + x_{ij})$. Modify the objective function to be equal to this new transportation cost. Some code to get you started:

```python
prod_cost = m.addVar(name='prod_cost')
m.addConstr(prod_cost ==..., name='set_prod_cost')
m.setObjective(prod_cost, GRB.MINIMIZE)
```
Again, resolve the model and see how the cost has changed.

In [5]:
# One solution
prod_cost = m.addVar(name='prod_cost')
m.addConstr(prod_cost == gp.quicksum(transp_cost[p,d] * gp.nlfunc.log(1 + x[p, d]) for p in production for d in distribution), name='set_prod_cost')
m.setObjective(prod_cost, GRB.MINIMIZE)
m.optimize()
print(f"The updated model had a total cost of {round(m.ObjVal,2)}")

The updated model had a total cost of 97.47


### Nonlinear expression 2

A very strange company rule stipulates that the following mathematical relationship must hold:

$ \log\Large(\frac{4 \cdot x_{\textrm{Baltimore,Indianapolis}}}{\sum_d x_{\textrm{Baltimore},d}})\normalsize \geq 0$

Add this constraint to the model using a nonlinear expression.

Hint: Start by adding a non-negative slack variable $s$ and writing the constraint as $s = \log\Large(\frac{4 \cdot x_{\textrm{Baltimore,Indianapolis}}}{\sum_d x_{\textrm{Baltimore},d}})$

Solve the model and see how the cost has changed.

In [6]:
# One solution
s = m.addVar(name="s")
NL2 = m.addConstr(s == gp.nlfunc.log(4 * x['Baltimore', 'Indianapolis'] / x.sum('Baltimore', '*')), name='strange_company_rule')
m.optimize()
print(f"The updated model had a total cost of {round(m.ObjVal,2)}")

The updated model had a total cost of 106.92


 ### Linearizing a nonlinear expression

Wait a minute...is the nonlinear constraint we just added unnecessarily complicated? Rewrite the nonlinear constraint using only linear expressions. Describe the purpose of the constraint in words.

Solve the model again and inspect the solution. Verify by inspection that it satisfies this newly added constraint.

Using the properties of logarithms, the constraint can be rewritten as:
$ x_{\textrm{Baltimore,Indianapolis}}\geq \frac 14\sum_d x_{\textrm{Baltimore},d}$

\begin{align*}
\log(\frac{4 \cdot x_{\textrm{Baltimore,Indianapolis}}}{\sum_d x_{\textrm{Baltimore},d}})\normalsize &\geq 0 \\
\frac{4 \cdot x_{\textrm{Baltimore,Indianapolis}}}{\sum_d x_{\textrm{Baltimore},d}} &\geq 1 \\
x_{\textrm{Baltimore,Indianapolis}}&\geq \frac 14\sum_d x_{\textrm{Baltimore},d}
\end{align*}

In [7]:
# One solution
# First, remove the complicated form of the constraint
m.remove(NL2)

# Meaning of the constraint: at least 25% of Baltimore's total production must be sent to Indianapolis
m.addConstr(x['Baltimore', 'Indianapolis'] >= 0.25 * x.sum('Baltimore', '*'), name='strange_company_rule_linearized')
m.optimize()
x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)

print(f"The updated model had a total cost of {round(m.ObjVal,2)}")
sol[sol.shipment > 0]

The updated model had a total cost of 106.92


cost    shipment
production  distribution                  
Baltimore   Indianapolis  5.09   38.666667
            Richmond      1.96  116.000000
Cleveland   Columbia      2.43   89.000000
            Nashville     4.13  101.000000
Little Rock St. Louis     2.92  121.000000
Birmingham  St. Louis     4.01   60.000000
Charleston  Indianapolis  2.61   56.333333
            Lexington     1.61  121.000000